# Creating a machine learning model from the data prep in BigG_AC 

**Two metholodogies will be implemented in the following models, one taking in consideration the model per truck, or one where the model takes into consideration the timestamps**

In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [2]:
Diagnostics_5246 = pd.read_csv('data/Diagnostics_5246.csv')
Diagnostics_1569 =  pd.read_csv('data/Diagnostics_1569.csv')

In [3]:
Diagnostics_5246

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,...,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,0
2,2,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,...,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,0
3,3,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,...,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,0
4,4,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,...,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,546659,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,...,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0
546660,546660,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,...,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,0
546661,546661,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,...,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,0
546662,546662,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,...,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,0


- Dropping the columns that will not be considered in the model

In [4]:
Diagnostics_5246 = Diagnostics_5246.drop(columns = ['Unnamed: 0', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'eventDescription', 'EventTimeStamp_DateOnly', 'LocationTimeStamp', 'LocationTimeStamp_DateOnly'])
Diagnostics_1569 = Diagnostics_1569.drop(columns = ['Unnamed: 0','ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource'])

Splitting the trucks between derate and non derate. For Diagnostics_5246

In [5]:
all_trucks = Diagnostics_5246['EquipmentID'].unique()
derate_trucks = Diagnostics_5246.loc[Diagnostics_5246['spn'] == 5246]['EquipmentID'].unique()
no_derate_trucks = all_trucks[np.isin(all_trucks, derate_trucks, invert=True)]

- put the two lists together (marking 1 for trucks with derate, 0 with non):

In [6]:
trucks_df = pd.concat([
            pd.DataFrame({'EquipmentID': derate_trucks, 'derate': 1}),
            pd.DataFrame({'EquipmentID': no_derate_trucks, 'derate': 0}) 
            ])

Using trest_train_split and “stratify” to ensure the proportions of derate/non-derate stay same in the samples:


- And then you’d use something like below to extract and train:

In [9]:
def test_split_data_custom(Diagnostics_5246, spn, percentage):
    all_trucks = Diagnostics_5246['EquipmentID'].unique()
    derate_trucks = Diagnostics_5246.loc[Diagnostics_5246['spn'] == spn]['EquipmentID'].unique()
    no_derate_trucks = all_trucks[np.isin(all_trucks, derate_trucks, invert=True)]


    #shuffle(sklearn) the array so that we get ramdom sequence in sample split
    all_Equip_s = shuffle(all_trucks, random_state=42)
    derate_Equip_s = shuffle(derate_trucks, random_state=42)
    no_derate_Equip_s = shuffle(no_derate_trucks, random_state=42)


    print(len(all_Equip_s))
    print(len(derate_Equip_s))
    print(len(no_derate_Equip_s))

    #convert to dataframe to locate rows
    df_all_Equip = pd.DataFrame(all_Equip_s, columns = ['EquipmentID'])
    df_derate_Equip = pd.DataFrame(derate_Equip_s, columns = ['EquipmentID'])
    df_no_derate_Equip = pd.DataFrame(no_derate_Equip_s, columns = ['EquipmentID'])


    #get the equipments % based on the passed percentage
    #keep_rows = int(total_rows * percentage)
    keep_rows_nde = int(len(no_derate_Equip_s) * percentage)
    print(keep_rows_nde)

    #============================================
    #STEP- 4 Get X1 for no_derate_Equip
    #============================================
    df_nde_pct_X1_train = df_no_derate_Equip.iloc[:keep_rows_nde, :]
    print(df_nde_pct_X1_train)

    #get rest of the percentage by doing not isin  lookup
    df_nde_pct_X1_test = df_no_derate_Equip[np.isin(df_no_derate_Equip,df_nde_pct_X1_train, invert=True)]
    print(df_nde_pct_X1_test) 

    #Make sure data is correct by checking we dont have equip id in both dataframes
    should_be_zero_nde= df_nde_pct_X1_train[np.isin(df_nde_pct_X1_train,df_nde_pct_X1_test)]
    print("should_be_zero nde = "  + str(should_be_zero_nde.size)) 

    #============================================
    #STEP 5 get X2 or train and Test
    #============================================
    keep_rows_de = int(len(derate_Equip_s) * percentage)
    print(keep_rows_de)

    df_de_pct_X2_train = df_derate_Equip.iloc[:keep_rows_de, :]
    print(df_de_pct_X2_train)

    #get rest of the percentage by doing not isin  lookup
    df_de_pct_X2_test = df_derate_Equip[np.isin(df_derate_Equip,df_de_pct_X2_train, invert=True)]
    print(df_de_pct_X2_test) 

    #Make sure data is correct by checking we dont have equip id in both dataframes
    should_be_zero_de= df_de_pct_X2_train[np.isin(df_de_pct_X2_train,df_de_pct_X2_test)]
    print("should_be_zero derate = "  + str(should_be_zero_de.size)) 


    # now we have the id ready for trian and test from both derate and no derate data frames. 
    # combine df_nde_pct_X1_train & df_de_pct_X2_train, combine df_nde_pct_X1_test and df_de_pct_X2_test
    combined_df_X_train = pd.concat([df_nde_pct_X1_train, df_de_pct_X2_train], ignore_index=True)
    print(combined_df_X_train)


    # combine df_nde_pct_X1_test & df_de_pct_X2_test, combine df_de_pct_X2_test and df_de_pct_X2_test
    combined_df_X_test = pd.concat([df_nde_pct_X1_test, df_de_pct_X2_test], ignore_index=True)
    print(combined_df_X_test)



    #STEP 6 - Get X_train, y_train and X_test. y_test 
    # Next filter the data from original dataframe for X, y

    df_train = Diagnostics_5246[Diagnostics_5246['EquipmentID'].isin(combined_df_X_train['EquipmentID'].tolist())]
    X_train= df_train.drop(columns=['target'])
    y_train = df_train['target']

    df_test = Diagnostics_5246[Diagnostics_5246['EquipmentID'].isin(combined_df_X_test['EquipmentID'].tolist())]
    X_test =df_test.drop(columns=['target'])
    y_test = df_test['target']

    # x_train, x_test, y_train, y_test

    return X_train, X_test, y_train, y_test

- Spliting the data to train and test to try to get a classifier model 

In [10]:
X_train, X_test, y_train, y_test = test_split_data_custom(Diagnostics_5246, 5246, 0.8)

1039
189
850
680
     EquipmentID
0           2011
1           1843
2           1457
3           2213
4           1427
..           ...
675         2027
676         2112
677         2377
678         2319
679         2247

[680 rows x 1 columns]
     EquipmentID
680         2148
681         2256
682         2128
683         2249
684         2037
..           ...
845         1460
846         1368
847         1583
848         1986
849         1765

[170 rows x 1 columns]
should_be_zero nde = 0
151
     EquipmentID
0           2089
1           1918
2           1490
3           1399
4           1539
..           ...
146         1827
147         1669
148         1591
149         1557
150         1985

[151 rows x 1 columns]
     EquipmentID
151         2143
152         1768
153         1797
154         1401
155         1501
156         1602
157         1582
158         2211
159         1477
160         1384
161         1686
162         1437
163         1604
164         1698
165         1907


In [11]:
y_train

0         0
1         0
3         0
4         0
5         0
         ..
546657    0
546660    0
546661    0
546662    0
546663    0
Name: target, Length: 436643, dtype: int64

In [12]:
# train a decision tree classifier on the training set
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

ValueError: could not convert string to float: '2015-02-21 10:47:13'

In [ ]:
# make predictions on the testing set
y_pred = classifier.predict(X_test)

In [ ]:
# Use the model to make predictions on the testing set
y_pred =classifier.predict(X_test)

In [ ]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
# Calculate the classification report
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
# evaluate the performance of the classifier using accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
# Create a DataFrame to store the predicted and actual values
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Plot the predicted and actual values using Pandas plot function
df.plot(kind='bar')
plt.show()

- Tring to improve the model with different classifier.

In [ ]:
pipe = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(hidden_layer_sizes = (2,),
                             activation = 'tanh',
                             max_iter = 10000))
    ]
)

In [ ]:
pipe.fit(X_train, y_train) 


In [ ]:
accuracy_2 = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy_2)